# 注意力機制 (Attention Mechanism)

## 學習目標
- 理解 Attention 的核心概念：Query, Key, Value
- 實作 Self-Attention 機制
- 學習 Multi-Head Attention
- 應用於語音識別任務（李宏毅 HW4）

## 對應課程
- [李宏毅 ML 2021 - Self-Attention](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [HW4: Speaker Recognition with Self-Attention](https://github.com/ga642381/ML2021-Spring/tree/main/HW04)

## Attention 發展歷程

```
Attention 演進
├── 2014: Seq2Seq Attention (Bahdanau)
├── 2015: Luong Attention (簡化版)
├── 2017: Transformer (Self-Attention)
├── 2018: BERT (雙向 Self-Attention)
└── 2020+: Vision Transformer, 各種變體
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 固定隨機種子
torch.manual_seed(42)
np.random.seed(42)

---
## Part 1: Attention 核心概念

### Query, Key, Value

Attention 機制可以用搜尋引擎來比喻：

| 概念 | 比喻 | 作用 |
|------|------|------|
| **Query (Q)** | 搜尋關鍵字 | 你想找什麼 |
| **Key (K)** | 網頁標題 | 用來匹配查詢 |
| **Value (V)** | 網頁內容 | 實際返回的資訊 |

### Scaled Dot-Product Attention

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

In [ ]:
# ========== Scaled Dot-Product Attention ==========

def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Scaled Dot-Product Attention
    
    Args:
        Q: Query tensor (batch, seq_len, d_k)
        K: Key tensor (batch, seq_len, d_k)
        V: Value tensor (batch, seq_len, d_v)
        mask: 可選的遮罩
    
    Returns:
        output: 注意力輸出
        attention_weights: 注意力權重
    """
    d_k = Q.size(-1)
    
    # 計算注意力分數: QK^T / sqrt(d_k)
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
    
    # 應用遮罩（如果有）
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    
    # Softmax 獲得注意力權重
    attention_weights = F.softmax(scores, dim=-1)
    
    # 加權求和
    output = torch.matmul(attention_weights, V)
    
    return output, attention_weights

# 測試
batch_size = 2
seq_len = 4
d_k = 8

Q = torch.randn(batch_size, seq_len, d_k)
K = torch.randn(batch_size, seq_len, d_k)
V = torch.randn(batch_size, seq_len, d_k)

output, attention = scaled_dot_product_attention(Q, K, V)

print(f"Q shape: {Q.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attention.shape}")
print(f"\n注意力權重 (第一個樣本):")
print(attention[0].numpy().round(3))

In [ ]:
# ========== 可視化 Attention Weights ==========

def visualize_attention(attention_weights, title="Attention Weights"):
    """
    可視化注意力權重
    """
    plt.figure(figsize=(8, 6))
    plt.imshow(attention_weights.detach().numpy(), cmap='viridis')
    plt.colorbar()
    plt.xlabel('Key Position')
    plt.ylabel('Query Position')
    plt.title(title)
    plt.show()

# 可視化第一個樣本的注意力
visualize_attention(attention[0], "Attention Weights (Sample 1)")

---
## Part 2: Self-Attention

Self-Attention: Q, K, V 都來自同一個輸入序列。

In [ ]:
# ========== Self-Attention Layer ==========

class SelfAttention(nn.Module):
    """
    Self-Attention 層
    
    輸入通過三個線性變換得到 Q, K, V
    """
    
    def __init__(self, embed_dim):
        super().__init__()
        self.embed_dim = embed_dim
        
        # Q, K, V 的線性投影
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        
        # 輸出投影
        self.W_o = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x, mask=None):
        """
        Args:
            x: (batch, seq_len, embed_dim)
        """
        # 線性投影
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # Scaled Dot-Product Attention
        output, attention_weights = scaled_dot_product_attention(Q, K, V, mask)
        
        # 輸出投影
        output = self.W_o(output)
        
        return output, attention_weights

# 測試
self_attn = SelfAttention(embed_dim=64)
x = torch.randn(2, 10, 64)  # (batch, seq_len, embed_dim)
output, attn = self_attn(x)

print(f"輸入: {x.shape}")
print(f"輸出: {output.shape}")

---
## Part 3: Multi-Head Attention

多頭注意力允許模型在不同的表示子空間中學習不同的注意力模式。

In [ ]:
# ========== Multi-Head Attention ==========

class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention
    
    將 embedding 分成多個 head，每個 head 獨立做 attention，
    最後 concatenate 再投影。
    """
    
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim 必須能被 num_heads 整除"
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Q, K, V 投影（一次性投影到所有 head）
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        
        # 輸出投影
        self.W_o = nn.Linear(embed_dim, embed_dim)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.shape
        
        # 線性投影
        Q = self.W_q(x)  # (batch, seq_len, embed_dim)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # 重塑為多頭: (batch, num_heads, seq_len, head_dim)
        Q = Q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Scaled Dot-Product Attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        output = torch.matmul(attention_weights, V)
        
        # 重塑回原始維度: (batch, seq_len, embed_dim)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.embed_dim)
        
        # 輸出投影
        output = self.W_o(output)
        
        return output, attention_weights

# 測試
mha = MultiHeadAttention(embed_dim=64, num_heads=8)
x = torch.randn(2, 10, 64)
output, attn = mha(x)

print(f"輸入: {x.shape}")
print(f"輸出: {output.shape}")
print(f"Attention shape: {attn.shape}  # (batch, heads, seq, seq)")

In [ ]:
# ========== 可視化多頭注意力 ==========

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, ax in enumerate(axes.flat):
    ax.imshow(attn[0, i].detach().numpy(), cmap='viridis')
    ax.set_title(f'Head {i+1}')
    ax.set_xlabel('Key')
    ax.set_ylabel('Query')

plt.suptitle('Multi-Head Attention Patterns', fontsize=14)
plt.tight_layout()
plt.show()

---
## Part 4: Transformer Encoder Block

完整的 Transformer Encoder 包含：
- Multi-Head Self-Attention
- Position-wise Feed-Forward Network
- Layer Normalization
- Residual Connections

In [ ]:
# ========== Position-wise Feed-Forward Network ==========

class FeedForward(nn.Module):
    """
    Position-wise Feed-Forward Network
    
    FFN(x) = max(0, xW_1 + b_1)W_2 + b_2
    """
    
    def __init__(self, embed_dim, ff_dim, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, ff_dim)
        self.fc2 = nn.Linear(ff_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
# ========== Transformer Encoder Block ==========

class TransformerEncoderBlock(nn.Module):
    """
    Transformer Encoder Block
    
    結構:
    x → LayerNorm → MHA → Dropout → + x (residual)
      → LayerNorm → FFN → Dropout → + x (residual)
    """
    
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        
        self.attention = MultiHeadAttention(embed_dim, num_heads, dropout)
        self.feed_forward = FeedForward(embed_dim, ff_dim, dropout)
        
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # Self-Attention with residual
        attn_output, attn_weights = self.attention(self.norm1(x), mask)
        x = x + self.dropout(attn_output)
        
        # Feed-Forward with residual
        ff_output = self.feed_forward(self.norm2(x))
        x = x + self.dropout(ff_output)
        
        return x, attn_weights

# 測試
encoder_block = TransformerEncoderBlock(
    embed_dim=64,
    num_heads=8,
    ff_dim=256,
    dropout=0.1
)

x = torch.randn(2, 10, 64)
output, attn = encoder_block(x)
print(f"Encoder Block 輸出: {output.shape}")

---
## Part 5: Speaker Recognition（李宏毅 HW4）

### 任務說明

**目標**：根據語音片段識別說話者

**架構**：
- 輸入：MFCC 特徵序列
- Conformer/Transformer Encoder
- 平均池化
- 分類器

In [ ]:
# ========== 模擬 Speaker Recognition 資料 ==========

def generate_speaker_data(n_samples=1000, max_seq_len=128, feat_dim=40, n_speakers=600):
    """
    生成類似 HW4 的 Speaker Recognition 資料
    
    實際 HW4:
    - 600 位說話者
    - 每個語音片段是變長的 MFCC 序列
    """
    np.random.seed(42)
    
    data = []
    labels = []
    
    for _ in range(n_samples):
        # 隨機長度
        seq_len = np.random.randint(32, max_seq_len + 1)
        # 隨機特徵
        x = np.random.randn(seq_len, feat_dim).astype(np.float32)
        # 隨機說話者
        speaker = np.random.randint(0, n_speakers)
        
        data.append(x)
        labels.append(speaker)
    
    return data, np.array(labels)

# 生成資料
speaker_data, speaker_labels = generate_speaker_data(n_samples=1000, n_speakers=600)

print(f"樣本數: {len(speaker_data)}")
print(f"說話者數: {len(set(speaker_labels))}")
print(f"序列長度範例: {[len(x) for x in speaker_data[:5]]}")

In [ ]:
# ========== Speaker Recognition Dataset ==========

class SpeakerDataset(Dataset):
    """
    Speaker Recognition Dataset
    
    處理變長序列
    """
    
    def __init__(self, data, labels, max_len=128):
        self.data = data
        self.labels = labels
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        
        # Padding 或截斷
        if len(x) > self.max_len:
            x = x[:self.max_len]
        elif len(x) < self.max_len:
            pad_len = self.max_len - len(x)
            x = np.pad(x, ((0, pad_len), (0, 0)), mode='constant')
        
        return torch.FloatTensor(x), torch.LongTensor([y]).squeeze()

# 建立 Dataset
dataset = SpeakerDataset(speaker_data, speaker_labels, max_len=128)
print(f"Dataset 大小: {len(dataset)}")
print(f"樣本形狀: {dataset[0][0].shape}")

In [ ]:
# ========== Speaker Recognition Model ==========

class SpeakerRecognitionModel(nn.Module):
    """
    Speaker Recognition 模型 (HW4 架構)
    
    架構:
    1. 線性投影層
    2. Positional Encoding
    3. Transformer Encoder 層
    4. 平均池化
    5. 分類器
    """
    
    def __init__(
        self,
        input_dim=40,
        embed_dim=256,
        num_heads=4,
        num_layers=3,
        ff_dim=1024,
        num_speakers=600,
        dropout=0.1,
        max_len=128
    ):
        super().__init__()
        
        # 輸入投影
        self.input_projection = nn.Linear(input_dim, embed_dim)
        
        # Positional Encoding
        self.pos_encoding = self._create_positional_encoding(max_len, embed_dim)
        
        # Transformer Encoder 層
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])
        
        # 分類器
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(embed_dim, num_speakers)
        )
        
        self.dropout = nn.Dropout(dropout)
    
    def _create_positional_encoding(self, max_len, embed_dim):
        """建立 Sinusoidal Positional Encoding"""
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        return nn.Parameter(pe.unsqueeze(0), requires_grad=False)
    
    def forward(self, x):
        # x: (batch, seq_len, input_dim)
        
        # 輸入投影
        x = self.input_projection(x)
        
        # 加入位置編碼
        x = x + self.pos_encoding[:, :x.size(1), :]
        x = self.dropout(x)
        
        # 通過 Encoder 層
        attention_weights = []
        for layer in self.encoder_layers:
            x, attn = layer(x)
            attention_weights.append(attn)
        
        # 平均池化
        x = x.mean(dim=1)  # (batch, embed_dim)
        
        # 分類
        logits = self.classifier(x)
        
        return logits, attention_weights

# 測試模型
model = SpeakerRecognitionModel(
    input_dim=40,
    embed_dim=256,
    num_heads=4,
    num_layers=3,
    num_speakers=600
)

x = torch.randn(4, 128, 40)  # (batch, seq_len, feat_dim)
logits, attns = model(x)

print(f"輸入: {x.shape}")
print(f"輸出: {logits.shape}")
print(f"參數量: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# ========== 訓練 Speaker Recognition 模型 ==========

# 準備資料
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 訓練配置
model = SpeakerRecognitionModel(
    input_dim=40,
    embed_dim=128,
    num_heads=4,
    num_layers=2,
    num_speakers=600
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
criterion = nn.CrossEntropyLoss()

# 訓練
history = {'train_loss': [], 'val_acc': []}

for epoch in range(20):
    # 訓練
    model.train()
    train_loss = 0
    
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        logits, _ = model(x_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    # 驗證
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            logits, _ = model(x_batch)
            pred = logits.argmax(dim=1)
            correct += (pred == y_batch).sum().item()
            total += y_batch.size(0)
    
    val_acc = correct / total
    
    scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Val Acc={val_acc:.4f}")

---
## Part 6: Attention 變體

### 常見 Attention 變體

In [ ]:
# ========== Additive (Bahdanau) Attention ==========

class AdditiveAttention(nn.Module):
    """
    Additive Attention (Bahdanau, 2014)
    
    score = v^T tanh(W_1 Q + W_2 K)
    """
    
    def __init__(self, query_dim, key_dim, hidden_dim):
        super().__init__()
        self.W_q = nn.Linear(query_dim, hidden_dim, bias=False)
        self.W_k = nn.Linear(key_dim, hidden_dim, bias=False)
        self.v = nn.Linear(hidden_dim, 1, bias=False)
    
    def forward(self, query, key, value):
        # query: (batch, q_len, query_dim)
        # key: (batch, k_len, key_dim)
        
        q = self.W_q(query)  # (batch, q_len, hidden)
        k = self.W_k(key)    # (batch, k_len, hidden)
        
        # 擴展維度以進行廣播
        q = q.unsqueeze(2)  # (batch, q_len, 1, hidden)
        k = k.unsqueeze(1)  # (batch, 1, k_len, hidden)
        
        scores = self.v(torch.tanh(q + k)).squeeze(-1)  # (batch, q_len, k_len)
        
        attention_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, value)
        
        return output, attention_weights

# 測試
additive_attn = AdditiveAttention(query_dim=64, key_dim=64, hidden_dim=32)
q = torch.randn(2, 5, 64)
k = torch.randn(2, 10, 64)
v = torch.randn(2, 10, 64)

out, attn = additive_attn(q, k, v)
print(f"Additive Attention 輸出: {out.shape}")

In [ ]:
# ========== Relative Positional Attention ==========

class RelativePositionalAttention(nn.Module):
    """
    Relative Positional Attention
    
    在計算 attention score 時考慮位置之間的相對距離
    """
    
    def __init__(self, embed_dim, num_heads, max_relative_position=32):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        self.W_o = nn.Linear(embed_dim, embed_dim)
        
        # 相對位置嵌入
        self.max_relative_position = max_relative_position
        num_embeddings = 2 * max_relative_position + 1
        self.relative_position_k = nn.Embedding(num_embeddings, self.head_dim)
        self.relative_position_v = nn.Embedding(num_embeddings, self.head_dim)
    
    def _get_relative_positions(self, seq_len):
        """生成相對位置索引"""
        positions = torch.arange(seq_len).unsqueeze(0) - torch.arange(seq_len).unsqueeze(1)
        positions = positions.clamp(-self.max_relative_position, self.max_relative_position)
        positions = positions + self.max_relative_position
        return positions
    
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        
        Q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.W_k(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.W_v(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # 標準 attention score
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # 相對位置貢獻
        relative_positions = self._get_relative_positions(seq_len).to(x.device)
        rel_k = self.relative_position_k(relative_positions)  # (seq, seq, head_dim)
        
        # 加入相對位置 score
        rel_scores = torch.einsum('bhqd,qkd->bhqk', Q, rel_k) / math.sqrt(self.head_dim)
        scores = scores + rel_scores
        
        attention_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.embed_dim)
        output = self.W_o(output)
        
        return output, attention_weights

# 測試
rel_attn = RelativePositionalAttention(embed_dim=64, num_heads=4)
x = torch.randn(2, 16, 64)
out, attn = rel_attn(x)
print(f"Relative Positional Attention 輸出: {out.shape}")

---
## 🎯 總結

### Attention 機制比較

| 類型 | 公式 | 特點 |
|------|------|------|
| **Dot-Product** | $QK^T/\sqrt{d}$ | 計算效率高 |
| **Additive** | $v^T\tanh(W_qQ+W_kK)$ | 更有表達力 |
| **Multi-Head** | 多個並行 attention | 捕捉不同模式 |
| **Relative Position** | 加入相對位置資訊 | 更好的序列建模 |

### Self-Attention 優缺點

| 優點 | 缺點 |
|------|------|
| 可平行計算 | O(n²) 記憶體複雜度 |
| 長距離依賴 | 缺乏位置資訊 |
| 可解釋性強 | 對小資料集易過擬合 |

### 李宏毅 HW4 技巧

```
Strong Baseline 達成方法:
1. Conformer 架構（結合 CNN 和 Transformer）
2. 適當的 embed_dim 和 num_heads
3. 資料增強（SpecAugment）
4. 學習率調度
5. Label Smoothing
```

### 下一步

前往 `sequence_modeling/transformer.ipynb` 學習完整的 Transformer 架構！

---
## 練習題

### 練習 1: 實作 Masked Self-Attention

**目標**: 實作用於 Decoder 的 Causal Mask，防止看到未來資訊

In [ ]:
# 練習 1: Masked Self-Attention (Causal Attention)

def create_causal_mask(seq_len):
    """
    創建因果遮罩，防止看到未來資訊
    
    返回一個下三角矩陣：
    [[1, 0, 0, 0],
     [1, 1, 0, 0],
     [1, 1, 1, 0],
     [1, 1, 1, 1]]
    """
    mask = torch.tril(torch.ones(seq_len, seq_len))
    return mask

class MaskedSelfAttention(nn.Module):
    """Causal Self-Attention for Decoder"""
    
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        super().__init__()
        self.mha = MultiHeadAttention(embed_dim, num_heads, dropout)
    
    def forward(self, x):
        seq_len = x.size(1)
        # 創建因果遮罩並擴展為 (1, 1, seq, seq) 以便廣播
        causal_mask = create_causal_mask(seq_len).to(x.device)
        causal_mask = causal_mask.unsqueeze(0).unsqueeze(0)
        return self.mha(x, mask=causal_mask)

# 測試
masked_attn = MaskedSelfAttention(embed_dim=64, num_heads=4)
x = torch.randn(2, 8, 64)  # (batch, seq_len, embed_dim)
output, attn_weights = masked_attn(x)

print(f"輸入: {x.shape}")
print(f"輸出: {output.shape}")

# 視覺化因果注意力權重
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(create_causal_mask(8).numpy(), cmap='Blues')
plt.title('Causal Mask')
plt.xlabel('Key Position')
plt.ylabel('Query Position')

plt.subplot(1, 2, 2)
plt.imshow(attn_weights[0, 0].detach().numpy(), cmap='viridis')
plt.title('Masked Attention Weights (Head 1)')
plt.xlabel('Key Position')
plt.ylabel('Query Position')
plt.colorbar()

plt.tight_layout()
plt.show()

print("\n觀察：注意力權重只關注當前位置及之前的位置（下三角形式）")

### 練習 2: 比較不同 Attention 頭數的效果

**目標**: 實驗不同頭數對模型性能的影響

In [ ]:
# 練習 2: 比較不同頭數

def experiment_num_heads(embed_dim=64, head_options=[1, 2, 4, 8]):
    """比較不同 attention 頭數的效果"""
    
    # 建立簡單的序列分類任務
    torch.manual_seed(42)
    n_samples = 500
    seq_len = 32
    n_classes = 10
    
    # 合成資料
    X = torch.randn(n_samples, seq_len, embed_dim)
    y = torch.randint(0, n_classes, (n_samples,))
    
    # 分割
    train_X, val_X = X[:400], X[400:]
    train_y, val_y = y[:400], y[400:]
    
    results = []
    
    for num_heads in head_options:
        print(f"\n=== num_heads = {num_heads} ===")
        
        # 建立模型
        model = nn.Sequential(
            TransformerEncoderBlock(embed_dim, num_heads, embed_dim * 4, dropout=0.1),
            nn.AdaptiveAvgPool1d(1),
        )
        classifier = nn.Linear(embed_dim, n_classes)
        
        optimizer = torch.optim.Adam(list(model.parameters()) + list(classifier.parameters()), lr=1e-3)
        criterion = nn.CrossEntropyLoss()
        
        # 訓練
        for epoch in range(30):
            model.train()
            out, _ = model[0](train_X)
            out = out.mean(dim=1)  # 平均池化
            loss = criterion(classifier(out), train_y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # 評估
        model.eval()
        with torch.no_grad():
            out, _ = model[0](val_X)
            out = out.mean(dim=1)
            pred = classifier(out).argmax(dim=1)
            acc = (pred == val_y).float().mean().item()
        
        results.append({'num_heads': num_heads, 'accuracy': acc})
        print(f"  Val Accuracy: {acc:.4f}")
    
    return results

# 執行實驗
results = experiment_num_heads()

# 視覺化
plt.figure(figsize=(8, 5))
plt.bar([str(r['num_heads']) for r in results], [r['accuracy'] for r in results])
plt.xlabel('Number of Attention Heads')
plt.ylabel('Validation Accuracy')
plt.title('Effect of Number of Attention Heads')
plt.grid(True, alpha=0.3)
plt.show()

print("\n結論：適當的頭數可以讓模型捕捉不同的注意力模式")